In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [3]:
import sys
sys.path.append("..")
from simdb.databaseModel import *
import simdb.databaseAPI as api

In [4]:
# db_path = 'test_WORKS.db'
db_path = '../examples/setup_databases/andrej_raw.db'

In [5]:
from sqlalchemy import or_, and_, or_

## For migration to databaseAPI:

## HowTo

### Load Entry Table

In [85]:
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s) # load whole table
table

,entry_id,path,owner,url,type,description
id,,,,,,
1,2017_06_28_GfM_SMmin_rnd_k33_26,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
2,2017_06_09_GfM_SMmin_2ub_k11_01,/home/andrejb/Research/SIMS/2017_06_09_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from mi...
3,2017_09_26_GfM_SMmin_rnd_k27_38,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
4,2017_06_09_GfM_SMmin_2ub_k48_01,/home/andrejb/Research/SIMS/2017_06_09_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from mi...
5,2017_04_27_G_2ub_k27_01_06,/home/andrejb/Research/SIMS/2017_04_27_G_2ub_k...,andrejb,None,None,Simulations of atomistic dimers for comparison...
6,2017_02_14_G_2ub_k33_02_03,/home/andrejb/Research/SIMS/2017_02_14_G_2ub_k...,andrejb,None,None,
7,2018_04_06_2xUb_unlinked_22,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,andrejb,None,None,Simulations of 2x CG monomer. Unlinked dimer.
8,2017_09_26_GfM_SMmin_rnd_k33_36,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
9,2017_07_21_2ub_k11_02_02,/home/andrejb/Research/SIMS/2017_07_21_2ub_k11...,andrejb,None,None,


In [6]:
# f = and_(Main.keywords.any(name="linker", value="k48"), Main.keywords.any(name="ff", value="MARTINI"))
f = Main.keywords.any(name="expansion_type", value="cg_min_rnd")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

,entry_id,path,owner,url,type,description
id,,,,,,
27,2017_09_26_GfM_SMmin_rnd_k48_39,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
28,2017_06_28_GfM_SMmin_rnd_k48_7,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
39,2017_09_26_GfM_SMmin_rnd_k48_40,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
42,2017_06_28_GfM_SMmin_rnd_k48_22,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
61,2017_09_26_GfM_SMmin_rnd_k48_35,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
71,2017_06_28_GfM_SMmin_rnd_k48_19,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
82,2017_09_26_GfM_SMmin_rnd_k48_49,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
88,2017_06_28_GfM_SMmin_rnd_k48_21,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
116,2017_09_26_GfM_SMmin_rnd_k48_38,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...


In [7]:
f = Main.keywords.any(name="linker", value="k48")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

,entry_id,path,owner,url,type,description
id,,,,,,
4,2017_06_09_GfM_SMmin_2ub_k48_01,/home/andrejb/Research/SIMS/2017_06_09_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from mi...
17,2017_08_01_2ub_k48_01_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,andrejb,None,None,
27,2017_09_26_GfM_SMmin_rnd_k48_39,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
28,2017_06_28_GfM_SMmin_rnd_k48_7,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
39,2017_09_26_GfM_SMmin_rnd_k48_40,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
42,2017_06_28_GfM_SMmin_rnd_k48_22,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
61,2017_09_26_GfM_SMmin_rnd_k48_35,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...
65,2017_08_01_2ub_k48_02_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,andrejb,None,None,
71,2017_06_28_GfM_SMmin_rnd_k48_19,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,andrejb,None,None,Simulations of atomistic dimers. Start from ra...


In [10]:
f = Main.keywords.any(name="temp", value="300")
with api.session_handler(db_path) as s:
    table = api.get_entry_table(s, apply_filter=f) # load whole table
table

,entry_id,path,owner,url,type,description
id,,,,,,
16,2017_04_26_M_ub1_300K,/home/andrejb/Research/SIMS/2017_04_26_M_ub1_300K,andrejb,None,None,UBQ monomer simulations at diff. temperatures ...


### Obtain data for certain entry

In [ ]:
entry_id = "2017_04_21_fat10_FL_exp_03"

In [ ]:
with api.session_handler(db_path) as s:
    details = api.get_entry_details(s, entry_id)
details

In [ ]:
s = api.connect_database(db_path)

In [ ]:
s.query(Main.entry_id).filter(Main.entry_id == entry_id).one()

### Select entries

In [ ]:
table[selectByTag(table, "test")] # select by tag

In [ ]:
table[selectByKeyword(table, "ff", "MARTINI")] # select by keyword

In [ ]:
# combinations of selections
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "1")
tags     = selectByTag(table, "test2")

table[ff & chainlen & tags]

In [ ]:
# another example
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "2")
linker1  = selectByKeyword(table, "linker", "k48")
linker2  = selectByKeyword(table, "linker", "k63")
number   = selectByKeyword(table, "number", "01")

table[ff & chainlen & (linker1 | linker2) & ~number]

### Working with groups

In [ ]:
# add some entries to a group
sel = table[selectByKeyword(table, "linker", "k48")]
[add_group(db_path, entry_id, "Group 1") for entry_id in sel.entry_id]

# a second group
sel = table[selectByKeyword(table, "ff", "MARTINI")]
[add_group(db_path, entry_id, "Group 2") for entry_id in sel.entry_id]

In [ ]:
# get entries of group 1
entry_group_table1 = get_entry_table(db_path, group_names=["Group 1"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table1.shape[0]))

# get entries of group 2
entry_group_table2 = get_entry_table(db_path, group_names=["Group 2"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table2.shape[0]))

In [ ]:
# get entries from both groups
# some entries might be in both groups
merged_table = get_entry_table(db_path, group_names=["Group 1", "Group 2"], columns=["entry_id", "path"])
print("Merge results in {} entries.".format(merged_table.shape[0]))

In [ ]:
# remove some etnries from Group 1
sel = table[selectByKeyword(table, "ff", "MARTINI")]
[remove_group(db_path, entry_id, "Group 1") for entry_id in sel.entry_id]

In [13]:
session = api.connect_database(db_path=db_path)
# groups = api.get_all_groups(session=session)

OperationalError: (sqlite3.OperationalError) no such column: main.priority [SQL: u'SELECT count(*) AS count_1 \nFROM (SELECT main.id AS main_id, main.entry_id AS main_entry_id, main.path AS main_path, main.owner AS main_owner, main.url AS main_url, main.type AS main_type, main.description AS main_description, main.created_on AS main_created_on, main.added_on AS main_added_on, main.updated_on AS main_updated_on, main.priority AS main_priority \nFROM main JOIN association_main_groups ON main.id = association_main_groups.main_id JOIN groups ON groups.id = association_main_groups.group_id \nWHERE groups.name = ?) AS anon_1'] [parameters: ('Group 1',)] (Background on this error at: http://sqlalche.me/e/e3q8)

### Usefull for backend / viewer

In [ ]:
# open database (in SQLAlchemy we open a session)
# s = openDatabase(db_path) 
# one should not keep the DB open

# better:
with api.session_handler(db_path) as s:
    keywords = api.get_all_keywords(s)  # get all keyword with the values they have
pd.Series(keywords)

In [ ]:
with session_handler(db_path) as s:
    groups = get_all_groups(s)  # get all groups in DB
groups

## Add, edit remove Metadata

In [ ]:
s = open_database(db_path)

In [ ]:
entry = s.query(Main).filter(Main.entry_id == entry_id).first()
# entry.keywords

In [ ]:
entry.meta.all()

In [ ]:
mg = entry.meta.filter_by(name="barostat").first()

In [ ]:
mg.entries.all()

In [ ]:
entry.groups

In [ ]:
s.close()

### Tags

In [ ]:
add_tag(db_path, entry_id, "my_tag")

In [ ]:
remove_tag(db_path, entry_id, "my_tag")

### Keywords

In [ ]:
# add two keywords to certain entry
add_keyword(db_path, entry_id, keyword1="bla", keyword2="blub")

In [ ]:
# alter keyword1 of entry
alter_keyword(db_path, entry_id, keyword1="my_value")

In [ ]:
# remove keyword2 if it is "blub" and force remove keyword1
remove_keyword(db_path, entry_id, keyword1=None, keyword2="blub")

### Meta information

In [ ]:
meta_group_name = "More Information"

In [ ]:
# add some information to entry
add_meta_group(db_path, entry_id, meta_group_name)
add_meta_data(db_path, entry_id, meta_group_name, 
              run="the best one", 
              importance="rather large", 
              version="quite new")

In [ ]:
alter_meta_data(db_path, entry_id, meta_group_name,
               run = "actually not that good")

In [ ]:
# remove some meta data entries from group
remove_meta_data(db_path, entry_id, meta_group_name,
                run=None,
                version="quite new")

# or remove the whole group
remove_meta_group(db_path, entry_id, meta_group_name)

In [ ]:
# check out changes
pd.DataFrame(get_entry_details(db_path, entry_id)['meta'])

### Running functions

## Deva

* Add tags/keywords to all simulations in pandas table
* Add simulations to DB from pandas table